# Coursework 1
**Replace CID in the file name with your CID**

# Outline


- [Task 1](#task-1): Linear regression and feature selection <a name="index-task-1"></a>
  - [(1.1)](#task-11) <a name="index-task-11"></a>
  - [(1.2)](#task-12) <a name="index-task-12"></a>
  - [(1.3)](#task-13) <a name="index-task-13"></a>
  - [(1.4)](#task-14) <a name="index-task-14"></a>
- [Task 2](#task-2): Non-linear regression with Kernel Ridge Regression <a name="index-task-2"></a>
  - [(2.1)](#task-21) <a name="index-task-21"></a>
  - [(2.2)](#task-22)  <a name="index-task-22"></a>
- [Task 3](#task-3): Classification with the Multi-Layer Perceptron <a name="index-task-3"></a>
  - [(3.1)](#task-31) <a name="index-task-31"></a>
  - [(3.2)](#task-32)  <a name="index-task-32"></a>



---



In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

rng = np.random.default_rng(0)

<a name="task-1"></a>

# Task 1: Linear regression and feature selection [(index)](#index-task-1)

<a name="task-11"></a>

## (1.1) [(index)](#index-task-11)

In [3]:
def visualise(x_test, y_test, beta, prediction_scheme):
    x_test_aug = np.hstack([np.ones((x_test.shape[0], 1)), x_test])
    prediction = prediction_scheme(x_test_aug, beta)

    plt.figure()
    plt.plot(x_test_aug, y_test, '+', markersize=10, label="data")
    plt.plot(x_test, prediction, '+', markersize=10, label="prediction")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.show()

In [5]:
def linear_prediction(x, beta):
    assert x.shape[1] == beta.shape
    return x @ beta

In [ ]:
def ls(x_train, y_train):

In [10]:
def garrote(x_train, y_train, lambd, n_iters=10000, step_size=10**(-7), c_huber=10**(-4)):

    assert x_train.shape[0] == y_train.shape[0], "x_train and y_train must have same size"

    n, p = x_train.shape
    XX = x_train.T @ x_train
    Xy = x_train.T @ y_train
    beta = np.zeros(p)
    beta_ls = np.linalg.inv(XX) @ Xy

    for i in range(n_iters):
        grad_c = grad_huber(beta, c_huber)
        grad_c[0] = 0

        grad = 0
        for j in range(p):
            grad += 2 * x_train[:, j].T @ beta_ls * (y_train[j] - x_train[:, j].T @ beta.T @ beta_ls - beta_ls[0]) + lambd * grad_c

        # grad = 2 * (X.T @ beta_ls)

        beta -= step_size * grad / p

    return beta

In [9]:
def huber(beta, c=10**(-6)):
    return np.where(np.abs(beta) <= c, (beta**2)/2, c * (np.abs(beta) - c/2))

def grad_huber(beta, c=10**(-6)):
    g = np.empty_like(beta)
    return np.where(np.abs(beta) < c, beta, c * np.sign(beta))

In [7]:
test_df = pd.read_csv('asteroid_observations_test.csv')
train_df = pd.read_csv('asteroid_observations_train.csv')

lambdas = np.logspace(-1, 7.6, 15)

   Absolute magnitude  Albedo  Number of observations  Observation arc length  \
0                18.1   0.041                      15                     2.0   
1                12.5   0.139                     349                  7208.0   
2                14.1   0.062                     705                 25309.0   
3                16.9   0.097                      91                  3805.0   
4                11.5   0.062                    1300                  9606.0   

   Orbital eccentricity  Orbital inclination  Orbital period  \
0              0.201590            11.975800        3.912298   
1              0.032746             8.793883       11.942668   
2              0.179314            27.433960        5.749452   
3              0.066114            13.813845        4.592982   
4              0.008824             5.595907       12.006096   

   Asteroid diameter Asteroid class  
0              1.573            MBA  
1             12.355            TJN  
2             

<a name="task-12"></a>

## (1.2) [(index)](#index-task-12)

<a name="task-13"></a>

## (1.3) [(index)](#index-task-13)

<a name="task-14"></a>

## (1.4) [(index)](#index-task-14)

<a name="task-2"></a>

# Task 2: Non-linear regression with Kernel Ridge Regression [(index)](#index-task-2)

<a name="task-21"></a>

## (2.1) [(index)](#index-task-21)

In [2]:
def ridge_est(x_train, y_train, lambd=0.1):
    """
    """

    m, n = x_train.shape
    assert y_train.shape[0] == m, "x_train and y_train must have same size"

    x_aug = np.hstack([np.ones((m, 1)), x_train])
    I = np.identity(n)
    I[0] = 0.0
    beta = np.linalg.inv(x_aug.T @ x_aug + lambd * np.eye(n)) @ x_aug.T @ y_train
    return beta

<a name="task-22"></a>

## (2.2) [(index)](#index-task-22)

<a name="task-3"></a>

# Task 3: Classification with the Multi-Layer Perceptron [(index)](#index-task-3)

<a name="task-31"></a>

## (3.1) [(index)](#index-task-31)

<a name="task-32"></a>

## (3.2) [(index)](#index-task-32)